In [2]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchtext.data.utils import get_tokenizer
from datasets import load_dataset
from collections import defaultdict
import pickle
import json
import re
from tqdm import tqdm

In [3]:
# -----------------------------
# 1. Load MS MARCO V1.1 training dataset
# -----------------------------

# This will stream the data, you don't have to download the full file
dataset = load_dataset("microsoft/ms_marco", "v1.1", split="train")  # or "validation"

In [5]:
# -----------------------------
# 2. Create triples of (query, relevant_docs, irrelevant_docs)
# -----------------------------

# 1. Build passage pool
passage_to_idx = {}
idx_to_passage = []

for row in tqdm(dataset, desc="Building passage pool..."):
    for p in row['passages']['passage_text']:
        if p not in passage_to_idx:
            passage_to_idx[p] = len(idx_to_passage)
            idx_to_passage.append(p)
num_passages = len(idx_to_passage)

# 2. Create triples and selected passages
triples = []
selected_passages = []

for row in tqdm(dataset, desc="Creating triples..."):
    query = row['query']
    passage_texts = row['passages']['passage_text']
    is_selected = row['passages']['is_selected']

    num_rels = len(passage_texts)
    if num_rels == 0:
        continue  # No passages for this query

    # Gold passage: any with is_selected==1
    gold_idxs = [i for i, flag in enumerate(is_selected) if flag == 1]
    if gold_idxs:
        selected_passages.append(passage_texts[gold_idxs[0]])
    else:
        selected_passages.append(None)

    # Relevant docs: all passages for this query (usually 1-10, can be 0, already filtered)
    relevant_passages = passage_texts

    # Build mask to exclude relevant passages
    rel_indices = [passage_to_idx[p] for p in passage_texts]
    mask = np.ones(num_passages, dtype=bool)
    mask[rel_indices] = False

    # Sample matching number of irrels
    if mask.sum() >= num_rels:
        irrel_indices = np.random.choice(np.where(mask)[0], num_rels, replace=False)
    else:
        irrel_indices = np.random.choice(np.where(mask)[0], num_rels, replace=True)
    irrelevant_passages = [idx_to_passage[i] for i in irrel_indices]

    triples.append((query, relevant_passages, irrelevant_passages))
    if len(triples) % 1000 == 0:
        break

# Save
with open("triples_full.pkl", "wb") as f:
    pickle.dump(triples, f)
with open("selected_passages.pkl", "wb") as f:
    pickle.dump(selected_passages, f)

# Sanity check
print("Example query:", triples[0][0])
print("Relevant docs:", triples[0][1])      # List of N passages (N varies)
print("Irrelevant docs:", triples[0][2])    # List of N passages (N = len(rels))
print("Selected gold doc:", selected_passages[0])


Creating triples...:   1%|          | 999/82326 [00:20<27:26, 49.39it/s]


Example query: what is rba
Relevant docs: ["Since 2007, the RBA's outstanding reputation has been affected by the 'Securency' or NPA scandal. These RBA subsidiaries were involved in bribing overseas officials so that Australia might win lucrative note-printing contracts. The assets of the bank include the gold and foreign exchange reserves of Australia, which is estimated to have a net worth of A$101 billion. Nearly 94% of the RBA's employees work at its headquarters in Sydney, New South Wales and at the Business Resumption Site.", "The Reserve Bank of Australia (RBA) came into being on 14 January 1960 as Australia 's central bank and banknote issuing authority, when the Reserve Bank Act 1959 removed the central banking functions from the Commonwealth Bank. The assets of the bank include the gold and foreign exchange reserves of Australia, which is estimated to have a net worth of A$101 billion. Nearly 94% of the RBA's employees work at its headquarters in Sydney, New South Wales and a

In [ ]:
# -------------------------------
# 1b. Load triples & selected docs from file
# -------------------------------

# Load triples
with open("triples_full.pkl", "rb") as f:
    triples = pickle.load(f)

with open("selected_passages.pkl", "rb") as g:
    selected_passages= pickle.load(g)

In [ ]:
# -------------------------------
# 1c. Tokenize triples
# -------------------------------

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9 ]+', '', text)
    return text.split()

tokenized_triples = []
for query, rel_docs, irrel_docs in tqdm(triples, desc="Tokenizing triples"):
    tokenized_query = preprocess(query)
    tokenized_rels = [preprocess(doc) for doc in rel_docs]
    tokenized_irrels = [preprocess(doc) for doc in irrel_docs]
    tokenized_triples.append((tokenized_query, tokenized_rels, tokenized_irrels))

In [9]:
# -----------------------------
# 2. Load Vocabulary & Pre-trained Embeddings
# -----------------------------
with open("vocab_new.json", "r", encoding="utf-8") as f:
    word_to_ix = json.load(f)

ix_to_word = {int(i): w for w, i in word_to_ix.items()}
vocab_size = len(word_to_ix)

embed_dim = 200  
state = torch.load("text8_cbow_embeddings.pth", map_location='cpu')  # Shape: [vocab_size, embed_dim]
embeddings = state["embeddings.weight"] 

assert embeddings.shape[0] == vocab_size, "Vocab size mismatch!"

In [10]:
# -----------------------------
# 3. CBOW Model
# -----------------------------
class CBOW(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embed_dim)
        self.linear = nn.Linear(embed_dim, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).mean(dim=1)
        return self.linear(embeds)

cbow_model = CBOW(vocab_size, embed_dim)
cbow_model.embeddings.weight.data.copy_(embeddings)
cbow_model.embeddings.weight.requires_grad = False 


In [ ]:
# -------------------------------
# 4. Embed queries, rel and irrel documents using pre-trained CBOW model
# -------------------------------

device = torch.device("cpu")
#torch.set_num_threads(12)

batch_size = 64  # adjust to taste; big enough to speed up, small enough to never threaten RAM

def process_and_save_embeddings(
    tokenized_triples, word_to_ix, cbow_model, 
    query_embeds_path, rel_doc_embeds_path, irrel_doc_embeds_path,
    batch_size=64
):
    query_embeds_batch = []
    rel_doc_embeds_batch = []
    irrel_doc_embeds_batch = []
    
    for i, (tokenized_query, tokenized_rels, tokenized_irrels) in enumerate(
        tqdm(tokenized_triples, desc="CBOW embedding + streaming", total=len(tokenized_triples))
    ):
        # Query: embeddings per token
        q_ids = [word_to_ix[t] for t in tokenized_query if t in word_to_ix]
        if q_ids:
            with torch.no_grad():
                q_vecs = cbow_model.embeddings(torch.tensor(q_ids))
            query_embeds_batch.append(q_vecs)  # [q_len, embed_dim]
        else:
            query_embeds_batch.append(torch.zeros(1, cbow_model.embeddings.embedding_dim))
        
        # Relevant docs: list of (doc_len, embed_dim)
        rel_embs = []
        for doc_tokens in tokenized_rels:
            doc_ids = [word_to_ix[t] for t in doc_tokens if t in word_to_ix]
            if doc_ids:
                with torch.no_grad():
                    doc_vecs = cbow_model.embeddings(torch.tensor(doc_ids))
                rel_embs.append(doc_vecs)
            else:
                rel_embs.append(torch.zeros(1, cbow_model.embeddings.embedding_dim))
        rel_doc_embeds_batch.append(rel_embs)

        # Irrelevant docs: list of (doc_len, embed_dim)
        irrel_embs = []
        for doc_tokens in tokenized_irrels:
            doc_ids = [word_to_ix[t] for t in doc_tokens if t in word_to_ix]
            if doc_ids:
                with torch.no_grad():
                    doc_vecs = cbow_model.embeddings(torch.tensor(doc_ids))
                irrel_embs.append(doc_vecs)
            else:
                irrel_embs.append(torch.zeros(1, cbow_model.embeddings.embedding_dim))
        irrel_doc_embeds_batch.append(irrel_embs)

        # Save every batch_size triples
        if (i + 1) % batch_size == 0 or (i + 1) == len(tokenized_triples):
            with open(query_embeds_path, 'ab') as fq:
                pickle.dump(query_embeds_batch, fq)
            with open(rel_doc_embeds_path, 'ab') as fr:
                pickle.dump(rel_doc_embeds_batch, fr)
            with open(irrel_doc_embeds_path, 'ab') as fi:
                pickle.dump(irrel_doc_embeds_batch, fi)

            # Free RAM
            query_embeds_batch.clear()
            rel_doc_embeds_batch.clear()
            irrel_doc_embeds_batch.clear()


# Usage example:
process_and_save_embeddings(
    tokenized_triples[:5120], word_to_ix, cbow_model,
    "query_embeds.pkl", "rel_doc_embeds.pkl", "irrel_doc_embeds.pkl",
    batch_size=batch_size
)

# # Usage:
# process_and_save_embeddings(
#     tokenized_triples, word_to_ix, cbow_model,
#     "query_embeds.pkl", "rel_doc_embeds.pkl", "irrel_doc_embeds.pkl",
#     batch_size=2048
# )

CBOW embedding + streaming: 100%|██████████| 5120/5120 [00:36<00:00, 141.12it/s]


In [ ]:
# -----------------------------
# 5. Load Embeddings
# -----------------------------

def load_all_batches(path):
    all_data = []
    with open(path, "rb") as f:
        while True:
            try:
                batch = pickle.load(f)
                all_data.extend(batch)  # for lists of tensors, this flattens batches
            except EOFError:
                break
    return all_data

# Load them all
query_embeds = load_all_batches("query_embeds.pkl")           # list of [query_len, embed_dim] tensors
rel_doc_embeds = load_all_batches("rel_doc_embeds.pkl")       # list of lists: each is [num_docs] of [doc_len, embed_dim] tensors
irrel_doc_embeds = load_all_batches("irrel_doc_embeds.pkl")

In [24]:
# -----------------------------
# 4. Define distance function & Triplet loss function
# -----------------------------

# Cosine similarity for calculation of cosine distance
def cosine_similarity(x, y):
    return F.cosine_similarity(x, y, dim=1)

# Triplet loss function - will compute the loss for a batch of triplets
def triplet_loss_function(query, relevant_doc, irrelevant_doc, distance_function, margin):
    rel_dist = distance_function(query, relevant_doc)         # (batch,)
    irrel_dist = distance_function(query, irrelevant_doc)     # (batch,)
    triplet_loss = torch.relu(rel_dist - irrel_dist + margin)
    return triplet_loss.mean()                                # Average over batch

In [19]:
# -----------------------------
# 5. Define Two Tower Model (QueryTower and DocTower)
# -----------------------------

class QueryTower(nn.Module):
    def __init__(self, embed_dim, hidden_dim, num_layers=1, rnn_type='gru'):
        super().__init__()
        if rnn_type == 'gru':
            self.rnn = nn.GRU(embed_dim, hidden_dim, num_layers, batch_first=True)
        elif rnn_type == 'lstm':
            self.rnn = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)
        else:
            raise ValueError("Unknown rnn_type: choose 'gru' or 'lstm'")

    def forward(self, x, lengths):
        # x: (batch, seq_len, embed_dim)
        packed = nn.utils.rnn.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        out, h = self.rnn(packed)
        if isinstance(h, tuple):  # LSTM
            h = h[0]
        return h[-1]  # (batch, hidden_dim)

class DocTower(nn.Module):
    def __init__(self, embed_dim, hidden_dim, num_layers=1, rnn_type='gru'):
        super().__init__()
        if rnn_type == 'gru':
            self.rnn = nn.GRU(embed_dim, hidden_dim, num_layers, batch_first=True)
        elif rnn_type == 'lstm':
            self.rnn = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)
        else:
            raise ValueError("Unknown rnn_type: choose 'gru' or 'lstm'")

    def forward(self, x, lengths):
        packed = nn.utils.rnn.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        out, h = self.rnn(packed)
        if isinstance(h, tuple):
            h = h[0]
        return h[-1]


In [20]:
# -----------------------------
# 6. Prepare dataset for DataLoader
# -----------------------------

def collate_fn(batch):
    q_seqs, r_seqs, i_seqs = zip(*batch)
    q_lens = [x.shape[0] for x in q_seqs]
    r_lens = [x.shape[0] for x in r_seqs]
    i_lens = [x.shape[0] for x in i_seqs]
    q_padded = pad_sequence(q_seqs, batch_first=True)
    r_padded = pad_sequence(r_seqs, batch_first=True)
    i_padded = pad_sequence(i_seqs, batch_first=True)
    return q_padded, r_padded, i_padded, q_lens, r_lens, i_lens

class TripleDataset(Dataset):
    def __init__(self, X_queries, X_rels, X_irrels):
        self.X_queries = X_queries    # list of N_query tensors
        self.X_rels = X_rels          # list of lists (variable-length)
        self.X_irrels = X_irrels      # list of lists (same variable-length)
        self.pairs = []
        for i in range(len(self.X_queries)):
            n = len(self.X_rels[i])
            for j in range(n):
                self.pairs.append((i, j))  # (query idx, doc pair idx)

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        q_idx, d_idx = self.pairs[idx]
        qry = self.X_queries[q_idx]
        rel = self.X_rels[q_idx][d_idx]
        irrel = self.X_irrels[q_idx][d_idx]
        return qry, rel, irrel


# Create the dataset and dataloader for batching
batch_size = 64
dataset = TripleDataset(query_embeds, rel_doc_embeds, irrel_doc_embeds)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)


NameError: name 'query_embeds' is not defined

In [ ]:
# -----------------------------
# 7. Train the model
# -----------------------------

# embed_dim defined in section 3.

# Define hyperparameters
hidden_dim = 128  # Dimension of the hidden state in RNNs (GRU/LSTM - can be adjusted)
margin = 0.2  # Margin for triplet loss (can be adjusted)
distance_function = cosine_similarity)

qry_tower = QueryTower(embed_dim, hidden_dim, rnn_type='gru')  # or 'lstm'
doc_tower = DocTower(embed_dim, hidden_dim, rnn_type='gru')  # or 'lstm'

optimizer = torch.optim.Adam(list(qry_tower.parameters()) + list(doc_tower.parameters()), lr=1e-3)
num_epochs = 5  # Set as needed

for epoch in range(num_epochs):
    epoch_loss = 0.0
    for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        qry_embeds, rel_embeds, irrel_embeds, q_lens, r_lens, i_lens = batch

        # For each sample in the batch, check if ANY of qry, rel, irrel is all zeros
        qry_zero_mask = torch.all(qry_embeds == 0, dim=(1,2))
        rel_zero_mask = torch.all(rel_embeds == 0, dim=(1,2))
        irrel_zero_mask = torch.all(irrel_embeds == 0, dim=(1,2))

        mask = ~(qry_zero_mask | rel_zero_mask | irrel_zero_mask)

        if mask.sum() == 0:
            continue  # All are bad samples

        qry_embeds   = qry_embeds[mask]
        rel_embeds   = rel_embeds[mask]
        irrel_embeds = irrel_embeds[mask]
        q_lens = [q_lens[i] for i in range(len(mask)) if mask[i]]
        r_lens = [r_lens[i] for i in range(len(mask)) if mask[i]]
        i_lens = [i_lens[i] for i in range(len(mask)) if mask[i]]

        qry_vecs    = qry_tower(qry_embeds, q_lens)
        rel_vecs    = doc_tower(rel_embeds, r_lens)
        irrel_vecs  = doc_tower(irrel_embeds, i_lens)

        loss = triplet_loss_function(
            qry_vecs, rel_vecs, irrel_vecs,
            distance_function=distance_function,
            margin=margin
        )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {epoch_loss / len(dataloader):.4f}")


# Save final model after all epochs are done
torch.save({
    'epoch': num_epochs,
    'qry_tower_state_dict': qry_tower.state_dict(),
    'doc_tower_state_dict': doc_tower.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': epoch_loss,  # from last epoch
}, "two_tower_final.pt")

Epoch 1/5: 100%|██████████| 1600/1600 [10:28<00:00,  2.55it/s]


Epoch 1/5 | Loss: 0.0916


Epoch 2/5: 100%|██████████| 1600/1600 [08:48<00:00,  3.03it/s]


Epoch 2/5 | Loss: 0.0322


Epoch 3/5: 100%|██████████| 1600/1600 [09:45<00:00,  2.73it/s]


Epoch 3/5 | Loss: 0.0166


Epoch 4/5: 100%|██████████| 1600/1600 [10:03<00:00,  2.65it/s]


Epoch 4/5 | Loss: 0.0088


Epoch 5/5: 100%|██████████| 1600/1600 [11:44<00:00,  2.27it/s]


Epoch 5/5 | Loss: 0.0055


In [21]:
# -----------------------------
# 8. Load the trained model
# -----------------------------
hidden_dim = 128
embed_dim = 200
# 1. Define your model classes (as in your code)
qry_tower = QueryTower(embed_dim, hidden_dim, num_layers=1, rnn_type='gru')
doc_tower = DocTower(embed_dim, hidden_dim, num_layers=1, rnn_type='gru')

# 2. Create optimizer with the same params as training
optimizer = torch.optim.Adam(list(qry_tower.parameters()) + list(doc_tower.parameters()), lr=1e-3)

# 3. Load checkpoint
checkpoint = torch.load("two_tower_final.pt", map_location="cpu")  # or map_location="cuda" if using GPU

# 4. Restore state dicts
qry_tower.load_state_dict(checkpoint['qry_tower_state_dict'])
doc_tower.load_state_dict(checkpoint['doc_tower_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# 5. (Optional) get other info
epoch = checkpoint['epoch']
last_loss = checkpoint['loss']


In [ ]:
# -----------------------------
# 9. Load MS MARCO V1.1 validation dataset, process into triples and tokenize & embed
# -----------------------------

# Load MS MARCO validation split
val_dataset = load_dataset("microsoft/ms_marco", "v1.1", split="validation")

# 1. Build passage pool (same as training)
passage_to_idx = {}
idx_to_passage = []
for row in tqdm(val_dataset, desc="Building passage pool..."):
    for p in row['passages']['passage_text']:
        if p not in passage_to_idx:
            passage_to_idx[p] = len(idx_to_passage)
            idx_to_passage.append(p)
num_passages = len(idx_to_passage)

# 2. Create triples and selected passages
triples_val = []
selected_passages_val = []

for row in tqdm(val_dataset, desc="Creating validation triples..."):
    query = row['query']
    passage_texts = row['passages']['passage_text']
    is_selected = row['passages']['is_selected']

    num_rels = len(passage_texts)
    if num_rels == 0:
        continue  # No passages for this query

    # Gold passage: any with is_selected==1
    gold_idxs = [i for i, flag in enumerate(is_selected) if flag == 1]
    if gold_idxs:
        selected_passages_val.append(passage_texts[gold_idxs[0]])
    else:
        selected_passages_val.append(None)

    # Relevant docs: all passages for this query
    relevant_passages = passage_texts

    # Build mask to exclude relevant passages
    rel_indices = [passage_to_idx[p] for p in passage_texts]
    mask = np.ones(num_passages, dtype=bool)
    mask[rel_indices] = False

    # Sample matching number of irrels
    if mask.sum() >= num_rels:
        irrel_indices = np.random.choice(np.where(mask)[0], num_rels, replace=False)
    else:
        irrel_indices = np.random.choice(np.where(mask)[0], num_rels, replace=True)
    irrelevant_passages = [idx_to_passage[i] for i in irrel_indices]

    triples_val.append((query, relevant_passages, irrelevant_passages))

# Save
with open("triples_val.pkl", "wb") as f:
    pickle.dump(triples_val, f)
with open("selected_passages_val.pkl", "wb") as f:
    pickle.dump(selected_passages_val, f)


# Load triples & selected docs
with open("triples_val.pkl", "rb") as f:
    triples_val = pickle.load(f)

with open("selected_passages_val.pkl", "rb") as g:
    selected_passages_val = pickle.load(g)

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9 ]+', '', text)
    return text.split()

tokenized_triples_val = []
for query, rel_docs, irrel_docs in tqdm(triples, desc="Tokenizing triples"):
    tokenized_query = preprocess(query)
    tokenized_rels = [preprocess(doc) for doc in rel_docs]
    tokenized_irrels = [preprocess(doc) for doc in irrel_docs]
    tokenized_triples_val.append((tokenized_query, tokenized_rels, tokenized_irrels))


batch_size = 64  # adjust to taste; big enough to speed up, small enough to never threaten RAM


def process_and_save_embeddings(
    tokenized_triples_val, word_to_ix, cbow_model, 
    query_embeds_path, rel_doc_embeds_path, irrel_doc_embeds_path,
    batch_size=64
):
    query_embeds_batch = []
    rel_doc_embeds_batch = []
    irrel_doc_embeds_batch = []
    
    for i, (tokenized_query, tokenized_rels, tokenized_irrels) in enumerate(
        tqdm(tokenized_triples_val, desc="CBOW embedding + streaming", total=len(tokenized_triples_val))
    ):
        # Query: embeddings per token
        q_ids = [word_to_ix[t] for t in tokenized_query if t in word_to_ix]
        if q_ids:
            with torch.no_grad():
                q_vecs = cbow_model.embeddings(torch.tensor(q_ids))
            query_embeds_batch.append(q_vecs)  # shape: [query_len, embed_dim]
        else:
            query_embeds_batch.append(torch.zeros(1, cbow_model.embeddings.embedding_dim))
        
        # Relevant docs: list of (doc_len, embed_dim)
        rel_embs = []
        for doc_tokens in tokenized_rels:
            doc_ids = [word_to_ix[t] for t in doc_tokens if t in word_to_ix]
            if doc_ids:
                with torch.no_grad():
                    doc_vecs = cbow_model.embeddings(torch.tensor(doc_ids))
                rel_embs.append(doc_vecs)
            else:
                rel_embs.append(torch.zeros(1, cbow_model.embeddings.embedding_dim))
        rel_doc_embeds_batch.append(rel_embs)

        # Irrelevant docs: list of (doc_len, embed_dim)
        irrel_embs = []
        for doc_tokens in tokenized_irrels:
            doc_ids = [word_to_ix[t] for t in doc_tokens if t in word_to_ix]
            if doc_ids:
                with torch.no_grad():
                    doc_vecs = cbow_model.embeddings(torch.tensor(doc_ids))
                irrel_embs.append(doc_vecs)
            else:
                irrel_embs.append(torch.zeros(1, cbow_model.embeddings.embedding_dim))
        irrel_doc_embeds_batch.append(irrel_embs)

        # Save every batch_size triples
        if (i + 1) % batch_size == 0 or (i + 1) == len(tokenized_triples):
            with open(query_embeds_path, 'ab') as fq:
                pickle.dump(query_embeds_batch, fq)
            with open(rel_doc_embeds_path, 'ab') as fr:
                pickle.dump(rel_doc_embeds_batch, fr)
            with open(irrel_doc_embeds_path, 'ab') as fi:
                pickle.dump(irrel_doc_embeds_batch, fi)

            query_embeds_batch.clear()
            rel_doc_embeds_batch.clear()
            irrel_doc_embeds_batch.clear()

# Usage example:
process_and_save_embeddings(
    tokenized_triples_val, word_to_ix, cbow_model,
    "query_embeds_val.pkl", "rel_doc_embeds_val.pkl", "irrel_doc_embeds_val.pkl",
    batch_size=64
)

CBOW embedding + streaming: 100%|██████████| 1000/1000 [00:06<00:00, 148.18it/s]

Example validation query: walgreens store sales average
Relevant docs: ['The average Walgreens salary ranges from approximately $15,000 per year for Customer Service Associate / Cashier to $179,900 per year for District Manager. Average Walgreens hourly pay ranges from approximately $7.35 per hour for Laboratory Technician to $68.90 per hour for Pharmacy Manager. Salary information comes from 7,810 data points collected directly from employees, users, and jobs on Indeed.', 'The average revenue in 2011 of a Starbuck Store was $1,078,000, up  from $1,011,000 in 2010.    The average ticket (total purchase) at domestic Starbuck stores in  No … vember 2007 was reported at $6.36.    In 2008, the average ticket was flat (0.0% change).', 'In fiscal 2014, Walgreens opened a total of 184 new locations and acquired 84 locations, for a net decrease of 273 after relocations and closings. How big are your stores? The average size for a typical Walgreens is about 14,500 square feet and the sales floo

In [15]:
# -----------------------------
# 9b. Load validation data embeddings
# -----------------------------

# Load them all
query_embeds_val = load_all_batches("query_embeds_val.pkl")           # list of [query_len, embed_dim] tensors
rel_doc_embeds_val = load_all_batches("rel_doc_embeds_val.pkl")       # list of lists: each is [num_docs] of [doc_len, embed_dim] tensors
irrel_doc_embeds_val = load_all_batches("irrel_doc_embeds_val.pkl")

val_data = []
for i in range(len(query_embeds_val)):
    q_embed = query_embeds_val[i]
    rel_embeds = rel_doc_embeds_val[i]         # List of (rel_len, embed_dim)
    irrel_embeds = irrel_doc_embeds_val[i]     # List of (irrel_len, embed_dim)
    val_data.append((q_embed, rel_embeds, irrel_embeds))


In [22]:
# -----------------------------
# 8. Evaluate the model using Recall@K
# -----------------------------

def evaluate_model(
    qry_tower, doc_tower, val_data, selected_passages_val, rel_doc_texts_val, irrel_doc_texts_val,
    distance_fn, K=1, device="cpu"
):
    qry_tower.eval()
    doc_tower.eval()
    num_correct = 0
    total = 0

    with torch.no_grad():
        for i, (q_embed, rel_embeds, irrel_embeds) in enumerate(val_data):
            # Skip queries with no gold passage
            gold_text = selected_passages_val[i]
            if gold_text is None:
                continue

            # Build all candidate docs and their texts
            candidate_embeds = rel_embeds + irrel_embeds
            candidate_texts = rel_doc_texts_val[i] + irrel_doc_texts_val[i]  # rels + irrels

            # Find gold doc index among candidates
            try:
                gold_idx = candidate_texts.index(gold_text)
            except ValueError:
                # Gold doc not among candidates
                continue

            all_doc_tensors = [doc.to(device) for doc in candidate_embeds]
            doc_lens = [doc.shape[0] for doc in all_doc_tensors]
            padded_docs = torch.nn.utils.rnn.pad_sequence(all_doc_tensors, batch_first=True)

            # Encode query
            q_input = q_embed.unsqueeze(0).to(device)
            q_len = [q_embed.shape[0]]
            q_vec = qry_tower(q_input, q_len)

            # Encode all docs in batch
            d_vecs = doc_tower(padded_docs, doc_lens)

            # Compute distances (query vs. all docs)
            sims = distance_fn(q_vec.repeat(len(doc_lens), 1), d_vecs)  # (num_candidates,)
            sorted_indices = torch.argsort(sims, descending=True)  # Smallest = most similar

            # Recall@K: Is gold doc in top K?
            if gold_idx in sorted_indices[:K]:
                num_correct += 1
            total += 1

    recall_at_k = num_correct / total if total > 0 else 0.0
    return recall_at_k



In [ ]:
rel_doc_texts_val = [rels for _, rels, _ in triples_val]
irrel_doc_texts_val = [irrels for _, _, irrels in triples_val]


recall = evaluate_model(
    qry_tower, doc_tower, val_data, selected_passages_val, rel_doc_texts_val, irrel_doc_texts_val,
    distance_fn=cosine_similarity, K=1, device='cuda' if torch.cuda.is_available() else 'cpu'
)
print(f"Recall@1: {recall:.4f}")

In [19]:
# -----------------------------
# 11. Inference: Encode documents and queries, find top-k relevant docs
# -----------------------------

def encode_documents(doc_tower, all_doc_embeds, all_doc_lens, device='cpu', batch_size=128):
    doc_tower.eval()
    all_vecs = []
    with torch.no_grad():
        for i in range(0, len(all_doc_embeds), batch_size):
            batch_embeds = all_doc_embeds[i:i+batch_size].to(device)
            batch_lens = all_doc_lens[i:i+batch_size]
            vecs = doc_tower(batch_embeds, batch_lens)  # Shape: (batch, hidden_dim)
            all_vecs.append(vecs.cpu())
    return torch.cat(all_vecs, dim=0)  # Shape: (num_docs, hidden_dim)

def encode_query(qry_tower, query_embed, query_len, device='cpu'):
    qry_tower.eval()
    with torch.no_grad():
        query_vec = qry_tower(query_embed.to(device), query_len)
    return query_vec.cpu()  # Shape: (1, hidden_dim)

def find_top_k(query_vec, doc_vecs, k=5, distance_fn=None, similarity=True):
    # query_vec: (1, hidden_dim), doc_vecs: (num_docs, hidden_dim)
    if distance_fn is None:
        # Default: cosine similarity
        sims = F.cosine_similarity(query_vec, doc_vecs)
        topk = torch.topk(sims, k)  # highest similarity
        indices = topk.indices.cpu().numpy()
        scores = topk.values.cpu().numpy()
        return indices, scores
    else:
        vals = distance_fn(query_vec.expand_as(doc_vecs), doc_vecs)
        if similarity:
            # Higher value = more similar (like cosine similarity)
            topk = torch.topk(vals, k)  # highest similarity
            indices = topk.indices.cpu().numpy()
            scores = topk.values.cpu().numpy()
            return indices, scores
        else:
            # Lower value = more similar (like distances)
            topk = torch.topk(-vals, k)  # lowest distance
            indices = topk.indices.cpu().numpy()
            scores = -topk.values.cpu().numpy()
            return indices, scores


In [13]:
# -----------------------------
# 12. Calculate all document embeddings and lengths
# -----------------------------

num_docs = 10  # Number of relevant docs per query

all_doc_texts = []  # To hold all relevant doc texts for padding
all_query_texts = []

for query, rels, irrels in triples:
    all_query_texts.append(query)
    # rels is the list of relevant doc texts for this query
    # We slice to num_docs to match the embedding logic, then pad if needed
    for doc in rels[:num_docs]:
        all_doc_texts.append(doc)
    # Pad if fewer than num_docs
    while len(rels) < num_docs:
        all_doc_texts.append("")
        rels.append("")  # So embedding and text padding always match

# Step 1: Gather your document embedding sequences and their lengths
all_doc_embeds_list = []
all_doc_lens_list = []

for batch in dataloader:
    rel_embeds, r_lens = batch[1], batch[4]  # rel_embeds: (batch, seq_len, embed_dim)
    for i in range(rel_embeds.shape[0]):
        all_doc_embeds_list.append(rel_embeds[i])  # (seq_len, embed_dim)
        all_doc_lens_list.append(r_lens[i])

# Step 2: Pad all embeddings to max seq_len
# pad_sequence wants a list of (seq_len, embed_dim), returns (max_seq_len, num_docs, embed_dim)
padded = pad_sequence(all_doc_embeds_list, batch_first=True)  # (num_docs, max_seq_len, embed_dim)

all_doc_embeds = padded  # (num_docs, max_seq_len, embed_dim)
all_doc_lens = torch.tensor(all_doc_lens_list)

print("all_doc_embeds shape:", all_doc_embeds.shape)
print("all_doc_lens shape:", all_doc_lens.shape)
print("all_doc_texts length:", len(all_doc_texts))
print("all_query_texts length:", len(all_query_texts))


all_doc_embeds shape: torch.Size([51200, 175, 200])
all_doc_lens shape: torch.Size([51200])
all_doc_texts length: 823260
all_query_texts length: 82326


In [ ]:
# -----------------------------
# 13. Encode documents and queries, find top-k relevant docs
# -----------------------------

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 1. Encode all documents
doc_vecs = encode_documents(doc_tower, all_doc_embeds, all_doc_lens, device=device)
#print("Encoded document vectors shape:", doc_vecs.shape)

# 2. Encode a sample query (here, just using first doc for demo)
query_embed = all_doc_embeds[0].unsqueeze(0)
query_len = all_doc_lens[0].unsqueeze(0)
query_vec = encode_query(qry_tower, query_embed, query_len, device=device)
#print("Encoded query vector shape:", query_vec.shape)

# 3. Retrieve top k relevant documents
k = 5
indices, scores = find_top_k(query_vec, doc_vecs, k=k)

print("Query: ", all_query_texts[0])
print("Top document matches:")
for rank, i in enumerate(indices):
    print(f"{rank+1}: {all_doc_texts[i]}")
    print(f"   (score: {scores[rank]:.4f})")


Query:  in animals somatic cells are produced by and gametic cells are produced by
Top document matches:
1: To get your learner's permit, you'll need to visit your local Alaska DMV office and: 1  Be at least 14 years old. 2  Have your parent/legal guardian complete a Parent/Guardian Consent for a Minor (Form 433). 3  Complete an Application for Alaska Driver License, Permit or Identification Card (Form 478). Unrestricted Alaska Driver's License. You are eligible for an unrestricted driver's license after you hold your provisional license for at least 6 months OR until you are 18 years old. If you decide to upgrade your driver's license before you turn 18 years old, you'll need to:
   (score: 0.7120)
2: Agar plates may also be indicator plates, in which the organisms are not selected on the basis of growth, but are instead distinguished by a color change in some colonies, typically caused by the action of an enzyme on some compound added to the medium. Some commonly used agar plate type

In [29]:
# -----------------------------
# 14. Custom Query Inference
# -----------------------------

# Hardcode your query here
custom_query = "what is rba"

# Tokenize and embed the custom query
tokenizer = get_tokenizer("basic_english")  # or any other tokenizer you prefer
tokenized_query = tokenizer(custom_query)  # returns List[int] or torch.Tensor

# Map tokens to ids, filtering out tokens not in vocab
q_ids = [word_to_ix[t] for t in tokenized_query if t in word_to_ix]

if q_ids:
    with torch.no_grad():
        q_vecs = cbow_model.embeddings(torch.tensor(q_ids))  # (seq_len, embed_dim)
else:
    q_vecs = torch.zeros(1, embed_dim)  # Fallback for empty/unknown queries)

# Pad to (1, seq_len, embed_dim) for model
query_embed = q_vecs.unsqueeze(0)  # (1, seq_len, embed_dim)
query_len = torch.tensor([q_vecs.shape[0]])

# Move to device if needed
query_embed = query_embed.to(device)
query_len = query_len.to(device)

# Encode the hardcoded query
query_vec = encode_query(qry_tower, query_embed, query_len, device=device)

# Retrieve top k relevant documents
k = 5
indices, scores = find_top_k(query_vec, doc_vecs, k=k)

print("Custom Query:", custom_query)
print("Top document matches:")
for rank, i in enumerate(indices):
    print(f"{rank+1}: {all_doc_texts[i]}")
    print(f"   (score: {scores[rank]:.4f})")


Custom Query: what is rba
Top document matches:
1: Although sintering does occur in loose powders, it is greatly enhanced by compacting the powder, and most commercial sintering is done on compacts. Compacting is generally done at room temperature, and the resulting compact is subsequently sintered at elevated temperature without application of pressure. Making ceramic, plastic and metal objects by heating a powder. Sintering by laser is one of the additive fabrication methods used in rapid prototyping and manufacturing. See laser sintering and 3D printing.
   (score: 0.6989)
2: 1 Such a building would run at an average of $8.56 million to complete. 2  This does not include acquisition of the land or any demolition costs, however. 3  The above figures place this construction at a $59 per square foot cost, though a national average of stands between $50 to $70 for most projects.
   (score: 0.6966)
3: The kilometre (International spelling as used by the International Bureau of Weights an

In [ ]:
# -----------------------------
# 15. Redis test
# -----------------------------

r = redis.Redis(host='localhost', port=6379, decode_responses=True)

r.set('foo', 'bar')
# True
r.get('foo')
# bar

